In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [5]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

In [6]:
from sklearn.linear_model import LogisticRegression

#### new input (emma data)

In [7]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [8]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [9]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [17]:
# combining the dataframe runs too slow so I'm going to reduce 2000 most common words
# to a different number

# Utility function to create a list of the x most common words.

def bag_of_words(text, num):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(num)]
    

In [18]:
# Set up the bags.
alicewords = bag_of_words(alice_doc, 200)
persuasionwords = bag_of_words(persuasion_doc, 200)


In [19]:
# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [11]:
# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

In [20]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [24]:
persuasionwords = bag_of_words(persuasion_doc, 200)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [25]:
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,day,mouse,very,will,queer,dinah,creature,feeling,uppercross,manage,...,shake,till,of,consequence,subject,wonder,would,game,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [26]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

### logistic regression

In [31]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

train = lr.fit(X_train, y_train)

In [32]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.6823266219239373


col_0,Austen,Carroll
row_0,,
Austen,1552,117
Carroll,735,278


### Challenge 0:
Recall that the logistic regression model's best performance on the test set was 93%. See what you can do to improve performance. Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires. Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.

In [128]:
#word_counts = word_counts.append(emma_bow,ignore_index=True,verify_integrity=True)
word_counts = word_counts.loc[:6986, :]

In [107]:
len(word_counts)

8656

In [45]:
word_counts.head(3)

,day,mouse,very,will,queer,dinah,creature,feeling,uppercross,manage,...,shake,till,of,consequence,subject,wonder,would,game,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll


In [110]:
# adding a feature - number of verbs in each sentence

def count_deps(verb_counts, sentencedf):

    for (this, guy) in sentencedf:
        # set / reset counter to 0
        count = 0 
        # look at each token in the sentence
        for token in this:
            if (token.dep_ =='ROOT') or (token.dep_ =='aux'):
                count +=1
            
        verb_counts.append(count)
        
    return verb_counts

verb_counts = []

order = [alice_sents, persuasion_sents, emma_sents]

for one in order:
    count_deps(verb_counts, one)


6987

In [130]:
word_counts["verb_count"] = verb_counts

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [131]:
word_counts.head(1)

,day,mouse,very,will,queer,dinah,creature,feeling,uppercross,manage,...,till,of,consequence,subject,wonder,would,game,text_sentence,text_source,verb_count
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,6


Let's add another couple of features - total # of words per sentence and number of unique parts of speech per sentence.

In [150]:
def count_words_and_pos(total_words, num_pos, sentencedf):
    
    for (this, guy) in sentencedf:
        # number of words
        allw = [token for token in this]
        # add # of total words to the array
        total_words.append(len(allw))
        
        #get the parts of speech of all the wrods in the sentence
        allpos = [token.pos_ for token in this]
        num_pos.append(len(set(allpos)))
               
    return 

total_w_counts = []
total_pos  = []

order = [alice_sents, persuasion_sents, emma_sents]

for one in order:
    count_words_and_pos(total_w_counts, total_pos, one)
    
word_counts["total_w_counts"] = total_w_counts
word_counts["unique_pos"] = total_pos

In [152]:
Y2 = word_counts['text_source']
X2 = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, 
                                                    Y2,
                                                    test_size=0.4,
                                                    random_state=0)

In [153]:
lr = LogisticRegression()

train = lr.fit(X_train2, y_train2)

In [156]:
print('\nTest set score:', lr.score(X_test2, y_test2))
lr_predicted = lr.predict(X_test2)
pd.crosstab(y_test2, lr_predicted)


Test set score: 0.9116279069767442


col_0,Austen,Carroll
text_source,,
Austen,2088,40
Carroll,207,460


In [157]:
from sklearn.model_selection import cross_val_score

In [158]:
scores = cross_val_score(lr, X_test2,y_test2,cv=10)

print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.90 (+/- 0.03)


# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [33]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [169]:
macbeth = gutenberg.raw('shakespeare-macbeth.txt')
macbeth = text_cleaner(macbeth)
macbethdoc = nlp(macbeth)

In [170]:
macbeth_sents = [[sent, "Shakespeare"] for sent in macbethdoc.sents]

In [172]:
to_add = bow_features(pd.DataFrame(macbeth_sents), common_words)
to_add.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500


,day,mouse,very,will,queer,dinah,creature,feeling,uppercross,manage,...,shake,till,of,consequence,subject,wonder,would,game,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Actus, Primus, .)",Shakespeare
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Scoena, Prima, .)",Shakespeare
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Thunder, and, Lightning, .)",Shakespeare
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Enter, three, Witches, .)",Shakespeare
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(1, .)",Shakespeare


In [183]:
# getting the verb count, word count, and unique pos count for Macbeth

these_verbs = []
count_deps(these_verbs, macbeth_sents)
to_add["verb_count"] = these_verbs

w_count = []
pos_count = []

count_words_and_pos(w_count, pos_count, macbeth_sents)
to_add['total_w_counts'] = w_count
to_add['unique_pos'] = pos_count

In [159]:
# making a new df just of Alice in wonderland

carroll = word_counts[word_counts['text_source'] == 'Carroll']

In [184]:
# combining the macbeth dataframe and the caroll

shakeycar = carroll.append(to_add,ignore_index=True, verify_integrity=True)

In [186]:
shakeycar['text_source'].nunique()

2

In [187]:
shakeycar.head(2)

,day,mouse,very,will,queer,dinah,creature,feeling,uppercross,manage,...,consequence,subject,wonder,would,game,text_sentence,text_source,verb_count,total_w_counts,unique_pos
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,6,67,11
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,4,63,11


In [189]:
X_sc = shakeycar.drop(['text_sentence','text_source'], 1)
y_sc = shakeycar['text_source'] 

X_train_sc, X_test_sc, y_train_sc, y_test_sc = train_test_split(X_sc, y_sc, test_size=0.4,
                                                               random_state=0)

In [190]:
train = lr.fit(X_train_sc, y_train_sc)

In [191]:
print('\nTest set score:', lr.score(X_test_sc, y_test_sc))
lr_predicted = lr.predict(X_test_sc)
pd.crosstab(y_test_sc, lr_predicted)


Test set score: 0.8476589797344515


col_0,Carroll,Shakespeare
text_source,,
Carroll,512,175
Shakespeare,43,701


In [193]:
scores = cross_val_score(lr, X_test_sc,y_test_sc,cv=10)

print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.85 (+/- 0.06)
